# Taking a look inside the ImageNet Dataset

This script is just to help one understand how the ImageNet dataset is stored and annotated.

ImageNet is a dataset of images (in JPEG format) that contain 1000 different objects. For each object, there are about 1000 different images. 

Many, though not all, image files contain a corresponding XML file that has annotation information such as a bounding box to explicitly outline where the object is located, or many independent objects in the same image.

In [ ]:
# import some tools
import json,os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import xml.etree.ElementTree as ET
import matplotlib.patches as patches
%matplotlib inline

In [ ]:
# load the json config file because it has the path to the filelist
json_fn = 'ilsvrc.json'
config = json.load(open(json_fn))
print(json.dumps(config,indent=3))

In [ ]:
# open the file list
filelist = config['data']['train_filelist']
file = open(filelist)
# read all the lines, there is one file path per line in the files
train_filelist = file.readlines()
print('total image files: ',len(train_filelist))
# choose one image to use as an example, you can change this index to choose another image
image_index = 4000
image_filename = train_filelist[image_index].strip()
print('plotting filename: ',image_filename)
image = mpimg.imread(image_filename)
print('data shape: ',image.shape)
plt.imshow(image)


In [ ]:
# convert JPEG filename to be the corresponding XML filename
xml_fn = image_filename.replace('Data','Annotations').replace('JPEG','xml')
# some images have no corresponding annotations XML file
if os.path.exists(xml_fn): 
    print('xml filename:',xml_fn,os.path.exists(xml_fn))
    for line in open(xml_fn):
        print(line.replace('\n',''))
else:
    print('no xml file found')
    xml_fn = None

In [ ]:
# read the XML file with Python's XML reader (XML is a bit antiquated)
tree = ET.parse(xml_fn)
root = tree.getroot()

# read image properties
img_size = root.find('size')
img_width = int(img_size.find('width').text)
img_height = int(img_size.find('height').text)
# img_depth = int(img_size.find('depth').text)

# there can be multiple objects per iamge
objs = root.findall('object')
# holder for the bouding box coordinates
bndbxs = []
# loop over the objects
for object in objs:
    # inside object, locate bouding box
    bndbox = object.find('bndbox')
    bndbxs.append([
    int(bndbox.find('ymin').text),
    int(bndbox.find('xmin').text),
    int(bndbox.find('ymax').text),
    int(bndbox.find('xmax').text)
    ])

print(bndbxs)

# plot image
f,a = plt.subplots(1)
a.imshow(image)

# show bounding boxes
for box in bndbxs:
    x0 = box[1]
    y0 = box[0]
    dx = box[3] - box[1]
    dy = box[2] - box[0]
    rect = patches.Rectangle((x0, y0), dx, dy, linewidth=1, edgecolor='r', facecolor='none')
    a.add_patch(rect)